In [ ]:
import tweepy
from twitter_authentication import API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, BEARER_TOKEN
import json

# V1 AUTH
#auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
#auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#tweepy.Client(bearer_token=BEARER_TOKEN, consumer_key=API_KEY, consumer_secret=API_SECRET, access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET)


In [ ]:
url_list = []

def save_list(vehicle_type):
    jsonString = json.dumps(url_list)
    path = "./dataset/" + vehicle_type + "/img_urls.json"
    jsonFile = open(path, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    

def read_list(vehicle_type: str):
    path = "./dataset/" + vehicle_type + "/img_urls.json"
    fileObject = open(path, "r")
    jsonContent = fileObject.read()
    try:
        url_list = json.loads(jsonContent)
    except:
        #json is empty
        url_list = []
        pass
    return url_list
    #print(url_list)

In [ ]:
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Users of interest: UAWeapons, OSINTua, RALee85, praisethesteph, 200_zoka, oryxspioenkop, Arslon_Xudosi
# from:twitterdev
# query_t72 = 'T-72 has:images -is:retweet (lang:en OR lang:ru OR lang:uk)'
# query_mtlb = '(MT-LB OR MT-LBV OR MT-LBVM OR MT-LBVMK OR MT-LBVM/K) -is:retweet from:UAWeapons'

query_vehicle_list = [  # (model,query content)
    ('MT-LB','MT-LB OR MT-LBV OR MT-LBVM OR MT-LBVMK OR MT-LBVM/K OR MT-LBu'),
    ('BTR-80','BTR-80 OR BTR-82'),
    ('BTR-82A','BTR-80A OR BTR-82A'),
    ('BMP-1','BMP-1'),
    ('BMP-2','BMP-2'),
    ('BMP-3','BMP-3 OR BMP-3M'),
    ('T-64','T-64 OR T-64A OR T-64B OR T-64BV OR T-64B1M OR T-64BM OR T-64BM2'),
    ('T-72','T-72 OR T-72A OR T-72AV OR T-72AMT OR T-72B OR T-72BA OR T-72B3 OR T-72B3 OR T-72M OR T-72M1'),
    ('T-80','T-80 OR T-80B OR T-80BV OR T-80U OR T-80BVM'),
    ('T-90','T-90 OR T-90A OR T-90M'),
    ('2S1','2S1 OR Gvozdika'),
    ('2S3','2S3 OR Akatsiya'),
    ('2S19','2S19 OR 2S19M OR 2S19M1 OR 2S19M2 OR Msta OR Msta-S OR Msta-SM2'),
    ('BM-21','BM-21'),
]

for vehicle in query_vehicle_list:
    model, query_content = vehicle
    
    url_counter = 0
    url_list = []
    url_list = read_list(model)
    
    query = '(' + query_content + ') (russian OR ukrainian OR ukraine OR russia OR army OR captured OR kherson OR kharkiv OR oblast OR donetsk OR severodonetsk OR luhansk OR lugansk OR Dnieper OR Dnipro OR izium OR izyum) has:images -is:retweet'
    #print(query)

    paginator = tweepy.Paginator(client.search_recent_tweets, query=query, max_results=100, limit=1000, expansions=['attachments.media_keys'], media_fields=['url'])
    for page in paginator:
        #print(page.includes['media'])  
        for item in page.includes['media']:
            url_list.append(item.url)
            url_counter += 1

    save_list(model)
    print('Saved', url_counter, model)


In [ ]:

#https://stackoverflow.com/questions/70854869/tweepy-problem-retrieving-username-informations-with-twitter-v2-api
#https://stackoverflow.com/questions/72016766/tweepy-only-lets-me-get-100-results-how-do-i-get-more-ive-read-about-paginati



In [ ]:
import urllib.request
import os

def download_img(url, vehicle, file_name):
    full_path = './dataset/' + vehicle + '/' + file_name + '.jpg'
    urllib.request.urlretrieve(url, full_path)

url_list2 = ["https://pbs.twimg.com/media/FcjBEEAXwAAxuU_.jpg", "https://pbs.twimg.com/media/FcjBEO4WAAEK9Z6.jpg", "https://pbs.twimg.com/media/FcjBEXJXwAAsnsh.jpg", "https://pbs.twimg.com/media/Fci0tJPXkAI1hHa.jpg", "https://pbs.twimg.com/media/FcioNgiWIAAN0h8.jpg", "https://pbs.twimg.com/media/FcioO3UWIAMR7NB.jpg", "https://pbs.twimg.com/media/FcioS_gWIAIyfAG.jpg", "https://pbs.twimg.com/media/FcioV72XkAA9T93.jpg", "https://pbs.twimg.com/media/Fchz1FRXkAQ0kEV.jpg", "https://pbs.twimg.com/media/Fchz1meXEAQZUgV.jpg", "https://pbs.twimg.com/media/Fchz14fWQAE5b0_.jpg", "https://pbs.twimg.com/media/Fchj4nJWAAU8F4_.jpg", "https://pbs.twimg.com/media/Fchj4nRXoAA6obb.jpg", "https://pbs.twimg.com/media/Fchj4nTX0AAe0PN.jpg", "https://pbs.twimg.com/media/Fchj4nJX0AEHHVh.jpg", "https://pbs.twimg.com/media/FcAScW5XoAEIuuV.png", "https://pbs.twimg.com/media/FcjBEEAXwAAxuU_.jpg", "https://pbs.twimg.com/media/FcjBEO4WAAEK9Z6.jpg", "https://pbs.twimg.com/media/FcjBEXJXwAAsnsh.jpg", "https://pbs.twimg.com/media/Fci0tJPXkAI1hHa.jpg", "https://pbs.twimg.com/media/FcioNgiWIAAN0h8.jpg", "https://pbs.twimg.com/media/FcioO3UWIAMR7NB.jpg", "https://pbs.twimg.com/media/FcioS_gWIAIyfAG.jpg", "https://pbs.twimg.com/media/FcioV72XkAA9T93.jpg", "https://pbs.twimg.com/media/Fchz1FRXkAQ0kEV.jpg", "https://pbs.twimg.com/media/Fchz1meXEAQZUgV.jpg", "https://pbs.twimg.com/media/Fchz14fWQAE5b0_.jpg", "https://pbs.twimg.com/media/FchoCe9acAQw_dp.jpg", "https://pbs.twimg.com/media/FchoCe9aIAMhbCt.jpg", "https://pbs.twimg.com/media/FchoCe-aMAAsAPE.jpg", "https://pbs.twimg.com/media/Fchj4nJWAAU8F4_.jpg", "https://pbs.twimg.com/media/Fchj4nRXoAA6obb.jpg", "https://pbs.twimg.com/media/Fchj4nTX0AAe0PN.jpg", "https://pbs.twimg.com/media/Fchj4nJX0AEHHVh.jpg", "https://pbs.twimg.com/media/FchjlW_WIAEjWN6.jpg", "https://pbs.twimg.com/media/FchjlodXoAErkL1.jpg", "https://pbs.twimg.com/media/Fchjl0NXgAEvpxD.jpg", "https://pbs.twimg.com/media/Fchjl9HWIAAljJL.jpg", "https://pbs.twimg.com/media/Fceuk_nXgAEopTW.jpg", "https://pbs.twimg.com/media/FceulRiX0AEXT6g.jpg", "https://pbs.twimg.com/media/FcXwl3xaAAY72_q.jpg", "https://pbs.twimg.com/media/FcXFyz_aUAIlm_T.jpg", "https://pbs.twimg.com/media/FcXFzqLaIAA_rak.jpg", "https://pbs.twimg.com/media/FcTuW_iWAAQzlnq.jpg", "https://pbs.twimg.com/media/FcPEto9XkAUzJI4.jpg", "https://pbs.twimg.com/media/FcPEtqqXwAMHfxO.jpg", "https://pbs.twimg.com/media/FcPEtqCXEAYiNkP.jpg", "https://pbs.twimg.com/media/FcPEtpoWAAAwWIK.jpg", "https://pbs.twimg.com/media/BwZYanYCcAA7A-A.jpg", "https://pbs.twimg.com/media/FcFHMVFXEAA5IRb.jpg", "https://pbs.twimg.com/media/FcFHQdxXgAAwOm5.jpg", "https://pbs.twimg.com/media/FcC_6V3WQAAdAJj.jpg", "https://pbs.twimg.com/media/FcAScW5XoAEIuuV.png", "https://pbs.twimg.com/media/FcHnL0UWAAAWY-B.png",]

counter = 0
for url in url_list2:
    counter += 1
    download_img(url, '2S1', str(counter))

In [ ]:
import os

def count_img(vehicle_type):
    counter = 0
    folder_path = './dataset/' + vehicle_type
    for path in os.scandir(folder_path):
        if path.is_file():
            if path.name[-4:] == '.jpg':
                counter += 1

    print(counter)

#count_img('2S3')

In [ ]:
import csv
#UNFINISHED
def csv_lookup(vehicle_type):
    with open('./dataset/test.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in csv_reader:
            _, _, system, status, url, _ = row
            if vehicle_type in system:
                url_list.append(url)
                print(row)
#UNFINISHED